(sec-mpi-collective)=
# 集合通信

{ref}`sec-mpi-point2point` 介绍了点对点通信，即发送方和接收方之间相互传输数据。本节主要介绍一种全局的通信方式：集合通信，即在一个组里的多个进程间同时传输数据。集合通信目前只有阻塞的方式。

常用的集合通信主要有以下几类：

* 同步，比如 [`Comm.Barrier`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Barrier)。
* 数据移动，比如 [`Comm.Bcast`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Bcast)， [`Comm.Scatter`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Scatter)，[`Comm.Gather`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Gather)等。
* 集合计算，比如 [`Comm.Reduce`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Reduce)， [`Intracomm.Scan`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Intracomm.html#mpi4py.MPI.Intracomm.Scan) 等。

首字母大写的函数是基于缓存的，比如 `Comm.Bcast`， `Comm.Scatter`，`Comm.Gather`，[`Comm.Allgather`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Allgather)， [`Comm.Alltoall`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Alltoall)。首字母小写的函数可以收发 Python 对象，比如 [`Comm.bcast`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.bcast)，[`Comm.scatter`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.scatter)，[`Comm.gather`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.gather)，[`Comm.allgather`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.allgather)， [`Comm.alltoall`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.alltoall)。

## 同步

MPI 的计算分布在多个进程，每个进程的计算速度有快有慢。`Comm.Barrier` 对 Communicator 里所有进程都执行同步等待，就像 Barrier 的英文名一样，设置一个屏障，等待所有进程都执行完。计算比较快的进程们达到 `Comm.Barrier`，无法执行 `Comm.Barrier` 之后的计算逻辑，必须等待其他所有进程都运行到 `Comm.Barrier` 才行。

## 数据移动

### 广播

`Comm.Bcast` 将数据从一个发送者全局广播给组里所有进程，广播操作适用于需要将同一份数据发送给所有进程的场景，例如将一个全局变量的值发送给所有进程，如 {numref}`fig-mpi-broadcast` 所示。

```{figure} ../img/ch-mpi/broadcast.svg
---
width: 600px
name: fig-mpi-broadcast
---
Broadcast
```
### 案例：广播

{numref}`code-mpi-broadcast-py` 演示了如何将一个 NumPy 数组广播到所有的进程：

```{code-block} python
:caption: broadcast.py
:name: code-mpi-broadcast-py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

comm.Barrier()

N = 5
if comm.rank == 0:
    A = np.arange(N, dtype=np.float64)    # rank 0 初始化数据到变量 A
else:
    A = np.empty(N, dtype=np.float64)     # 其他节点的变量 A 为空

# 广播
comm.Bcast([A, MPI.DOUBLE])

# 验证所有节点上的 A
print("Rank:%2d, data:%s" % (comm.rank, A))
```

In [2]:
!mpiexec -np 4 python broadcast.py

Rank: 0, data:[0. 1. 2. 3. 4.]
Rank: 2, data:[0. 1. 2. 3. 4.]
Rank: 1, data:[0. 1. 2. 3. 4.]
Rank: 3, data:[0. 1. 2. 3. 4.]


### Scatter 和 Gather

`Comm.Scatter` 和 `Comm.Gather` 是一组相对应的操作。

* `Comm.Scatter` 将数据从一个进程分散到组中的所有进程，一个进程将数据分散成多个块，每个块发送给对应的进程。其他进程接收并存储各自的块。Scatter 操作适用于将一个较大的数据集分割成多个小块。
* `Comm.Gather` 与 `Comm.Scatter` 相反，将组里所有进程的小数据块归集到一个进程上。

```{figure} ../img/ch-mpi/scatter-gather.svg
---
width: 600px
name: mpi-scatter-gather
---
Scatter 与 Gather
```

### 案例：Scatter

{numref}`code-mpi-scatter` 演示了如何使用 Scatter 将数据分散到所有进程。

```{code-block} python
:caption: scatter.py
:name: code-mpi-scatter

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.empty([size, 8], dtype='i')
    sendbuf.T[:,:] = range(size)
    print(f"Rank: {rank}, to be scattered: \n{sendbuf}")
recvbuf = np.empty(8, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
print(f"Rank: {rank}, after scatter: {recvbuf}")
assert np.allclose(recvbuf, rank)
```

In [3]:
!mpiexec -np 4 python scatter.py

Rank: 0, to be scattered: 
[[0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3]]
Rank: 0, after scatter: [0 0 0 0 0 0 0 0]
Rank: 1, after scatter: [1 1 1 1 1 1 1 1]
Rank: 2, after scatter: [2 2 2 2 2 2 2 2]
Rank: 3, after scatter: [3 3 3 3 3 3 3 3]


### Allgather 和 Alltoall

另外两个比较复杂的操作是 `Comm.Allgather` 和 `Comm.Alltoall`。

`Comm.Allgather` 是 `Comm.Gather` 的进阶版，如 {numref}`fig-mpi-allgather` 所示，它把散落在多个进程的多个小数据块发送给每个进程，每个进程都包含了一份相同的数据。

```{figure} ../img/ch-mpi/allgather.svg
---
width: 600px
name: fig-mpi-allgather
---
Allgather
```

`Comm.Alltoall` 是 `Comm.Scatter` 和 `Comm.Gather` 的组合，如 {numref}`fig-mpi-alltoall` 所示，先进行 `Comm.Scatter`，再进行 `Comm.Gather`。如果把数据看成一个矩阵，`Comm.Alltoall` 又可以被看做是一种全局的转置（Transpose）操作。

```{figure} ../img/ch-mpi/alltoall.svg
---
width: 600px
name: fig-mpi-alltoall
---
Alltoall
```

## 集合计算

集合计算是散落在不同进程的数据聚合在一起的同时对数据进行计算，比如 `Comm.Reduce`。如 {numref}`fig-mpi-reduce` 和 {numref}`fig-mpi-scan` 所示，数据归集到某个进程时，还执行了聚合函数 `f`，常用的聚合函数有求和 [`MPI.SUM`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.SUM.html) 等。

```{figure} ../img/ch-mpi/reduce.svg
---
width: 500px
name: fig-mpi-reduce
---
Reduce
```

```{figure} ../img/ch-mpi/scan.svg
---
width: 500px
name: fig-mpi-scan
---
Scan
```